# Gold Layer: Transform Silver to Gold Analytics Tables

## Purpose
This notebook creates business-ready aggregated datasets from Silver layer tables for analytics and reporting.

## Inputs
- **Source Lakehouse**: `tm2020_silver`
- **Source Tables**:
  - `silver_replays`: Replay metadata
  - `silver_ghost_samples`: Telemetry samples
  - `silver_maps`: Map dimension
  - `silver_players`: Player dimension

## Outputs
- **Target Lakehouse**: `tm2020_gold`
- **Tables**:
  - `gold_player_stats`: Aggregate player performance metrics
  - `gold_map_leaderboard`: Best times per map
  - `gold_race_analytics`: Race performance analysis
  - `gold_checkpoint_analysis`: Checkpoint-level insights

## Business Metrics
- Player statistics: total races, average times, best times, consistency
- Leaderboards: rankings per map, global rankings
- Race analytics: speed profiles, checkpoint performance
- Trend analysis: performance over time

In [ ]:
# Initialize Spark session and imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, count, min, max, avg, sum as spark_sum, stddev,
    row_number, rank, dense_rank, collect_list,
    current_timestamp, to_date
)
from pyspark.sql.window import Window

# Initialize Spark session (pre-configured in Fabric)
spark = SparkSession.builder.appName("Gold_Aggregation").getOrCreate()

print("Spark session initialized")
print(f"Spark version: {spark.version}")

In [ ]:
# Define table names
# TODO: Ensure both Silver and Gold Lakehouses are attached to this notebook
silver_replays = "tm2020_silver.silver_replays"
silver_ghost_samples = "tm2020_silver.silver_ghost_samples"
silver_maps = "tm2020_silver.silver_maps"
silver_players = "tm2020_silver.silver_players"

gold_player_stats = "gold_player_stats"
gold_map_leaderboard = "gold_map_leaderboard"
gold_race_analytics = "gold_race_analytics"
gold_checkpoint_analysis = "gold_checkpoint_analysis"

print("Silver sources configured")
print("Gold targets configured")

In [ ]:
# Read from Silver layer
# TODO: Implement incremental processing for Gold layer

try:
    df_replays = spark.table(silver_replays)
    df_ghost_samples = spark.table(silver_ghost_samples)
    df_maps = spark.table(silver_maps)
    df_players = spark.table(silver_players)
    
    print(f"Replays: {df_replays.count()} records")
    print(f"Ghost samples: {df_ghost_samples.count()} records")
    print(f"Maps: {df_maps.count()} records")
    print(f"Players: {df_players.count()} records")
    
except Exception as e:
    print(f"Error reading Silver tables: {e}")
    print("Ensure Silver transformation notebook has been run successfully")

In [ ]:
# Create Gold Player Statistics table
# TODO: Add more advanced metrics (consistency, improvement trends)

try:
    df_gold_player_stats = df_replays.groupBy("player_login").agg(
        count("replay_id").alias("total_races"),
        count(col("map_uid").isNotNull()).alias("total_maps_played"),
        avg("race_time_ms").alias("avg_race_time_ms"),
        min("race_time_ms").alias("best_race_time_ms"),
        spark_sum("num_respawns").alias("total_respawns"),
        max("ingestion_date").alias("last_race_date")
    )
    
    # Join with player dimension for nickname
    df_gold_player_stats = df_gold_player_stats.join(
        df_players.select("player_login", "player_nickname"),
        "player_login",
        "left"
    ).select(
        "player_login",
        "player_nickname",
        "total_races",
        "total_maps_played",
        "avg_race_time_ms",
        "best_race_time_ms",
        "total_respawns",
        "last_race_date"
    )
    
    print(f"Gold player stats: {df_gold_player_stats.count()} records")
    df_gold_player_stats.show(5, truncate=False)
    
except Exception as e:
    print(f"Error creating gold_player_stats: {e}")

In [ ]:
# Create Gold Map Leaderboard table
# TODO: Handle ties in rankings, add percentile rankings

try:
    # Get best time per player per map
    df_best_times = df_replays.groupBy("player_login", "map_uid").agg(
        min("race_time_ms").alias("best_time_ms"),
        max("ingestion_date").alias("race_date")
    )
    
    # Add rankings per map
    window_spec = Window.partitionBy("map_uid").orderBy(col("best_time_ms").asc())
    df_gold_leaderboard = df_best_times.withColumn(
        "rank",
        rank().over(window_spec)
    )
    
    # Join with map and player dimensions
    df_gold_leaderboard = df_gold_leaderboard \
        .join(df_maps.select("map_uid", "map_name"), "map_uid", "left") \
        .join(df_players.select("player_login", "player_nickname"), "player_login", "left") \
        .select(
            "map_uid",
            "map_name",
            "player_login",
            "player_nickname",
            "best_time_ms",
            "rank",
            "race_date"
        )
    
    print(f"Gold map leaderboard: {df_gold_leaderboard.count()} records")
    df_gold_leaderboard.filter(col("rank") <= 10).show(10, truncate=False)
    
except Exception as e:
    print(f"Error creating gold_map_leaderboard: {e}")

In [ ]:
# Create Gold Race Analytics table
# TODO: Add more telemetry-based metrics (trajectory analysis, cornering speed)

try:
    # Calculate speed statistics per replay from ghost samples
    df_speed_stats = df_ghost_samples.groupBy("replay_id").agg(
        avg("speed").alias("avg_speed"),
        max("speed").alias("max_speed"),
        stddev("speed").alias("speed_stddev")
    )
    
    # Join with replay metadata
    df_gold_race_analytics = df_replays \
        .join(df_speed_stats, "replay_id", "left") \
        .select(
            "replay_id",
            "player_login",
            "map_uid",
            "race_time_ms",
            "num_respawns",
            "avg_speed",
            "max_speed",
            "speed_stddev",
            "ingestion_date"
        )
    
    print(f"Gold race analytics: {df_gold_race_analytics.count()} records")
    df_gold_race_analytics.show(5, truncate=False)
    
except Exception as e:
    print(f"Error creating gold_race_analytics: {e}")

In [ ]:
# Create Gold Checkpoint Analysis table (placeholder)
# TODO: Extract checkpoint times from metadata and analyze
# This requires parsing the checkpoints array from replays

try:
    # This is a placeholder - actual implementation requires
    # extracting checkpoint arrays and analyzing them
    # For now, create basic statistics from available data
    
    df_gold_checkpoint = df_replays.groupBy("map_uid").agg(
        count("replay_id").alias("total_races"),
        avg("num_checkpoints").alias("avg_checkpoints"),
        avg("race_time_ms").alias("avg_race_time_ms")
    ).join(
        df_maps.select("map_uid", "map_name"),
        "map_uid",
        "left"
    ).select(
        "map_uid",
        "map_name",
        "total_races",
        "avg_checkpoints",
        "avg_race_time_ms"
    )
    
    print(f"Gold checkpoint analysis: {df_gold_checkpoint.count()} records")
    df_gold_checkpoint.show(5, truncate=False)
    
    print("\nNote: Full checkpoint analysis requires extracting checkpoint times from metadata.checkpoints array")
    print("This is a placeholder showing basic map-level statistics")
    
except Exception as e:
    print(f"Error creating gold_checkpoint_analysis: {e}")

In [ ]:
# Write to Gold Delta tables
# TODO: Implement incremental refresh strategy
# For now using overwrite mode - optimize later

try:
    # Write gold_player_stats
    df_gold_player_stats.write \
        .format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .saveAsTable(gold_player_stats)
    print(f"Successfully wrote to {gold_player_stats}")
    
    # Write gold_map_leaderboard
    df_gold_leaderboard.write \
        .format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .saveAsTable(gold_map_leaderboard)
    print(f"Successfully wrote to {gold_map_leaderboard}")
    
    # Write gold_race_analytics
    df_gold_race_analytics.write \
        .format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .saveAsTable(gold_race_analytics)
    print(f"Successfully wrote to {gold_race_analytics}")
    
    # Write gold_checkpoint_analysis
    df_gold_checkpoint.write \
        .format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .saveAsTable(gold_checkpoint_analysis)
    print(f"Successfully wrote to {gold_checkpoint_analysis}")
    
    print("\nAll Gold tables written successfully!")
    
except Exception as e:
    print(f"Error writing Gold tables: {e}")

In [ ]:
# Gold layer summary report
# TODO: Create comprehensive analytics summary

try:
    print("=== Gold Layer Summary Report ===")
    
    # Player stats summary
    df_player_stats_check = spark.table(gold_player_stats)
    print(f"\ngold_player_stats: {df_player_stats_check.count()} players")
    print("Top 5 players by total races:")
    df_player_stats_check.orderBy(col("total_races").desc()).show(5, truncate=False)
    
    # Leaderboard summary
    df_leaderboard_check = spark.table(gold_map_leaderboard)
    print(f"\ngold_map_leaderboard: {df_leaderboard_check.count()} entries")
    print("Sample top 5 entries:")
    df_leaderboard_check.filter(col("rank") <= 5).show(5, truncate=False)
    
    # Race analytics summary
    df_race_analytics_check = spark.table(gold_race_analytics)
    print(f"\ngold_race_analytics: {df_race_analytics_check.count()} races")
    print("Average statistics:")
    df_race_analytics_check.select(
        avg("race_time_ms").alias("avg_race_time"),
        avg("avg_speed").alias("avg_speed"),
        avg("max_speed").alias("avg_max_speed")
    ).show()
    
    # Checkpoint analysis summary
    df_checkpoint_check = spark.table(gold_checkpoint_analysis)
    print(f"\ngold_checkpoint_analysis: {df_checkpoint_check.count()} maps analyzed")
    df_checkpoint_check.show(5, truncate=False)
    
except Exception as e:
    print(f"Error running summary report: {e}")